In [65]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pycaret.regression import *
from gui import WyszukiwarkaNieruchomosciGUI
import tkinter as tk

In [66]:
data = pd.read_csv('./combined_24.csv', index_col=0)
data_col = data.columns
columns_to_check_duplicates = data.columns.difference(['link'])
data0 = data[columns_to_check_duplicates].drop_duplicates()
data0 = data0.join(data['link'])[data_col]
len(data0[data0.cena.isna()])

50

In [67]:
many_nans = (data0.isna().sum(axis=1)/data0.shape[1]).sort_values(ascending = False)
many_nans_ids = many_nans[many_nans > 0.7].index
data0.drop(many_nans_ids, inplace=True)
l = len(data0[data0.cena.isna()])
data0['cena'].fillna(0, inplace=True)
l

44

In [68]:
root = tk.Tk()
app = WyszukiwarkaNieruchomosciGUI(root)
root.mainloop()

wanted_lok = app.saved_lokalizacja

try:
    wanted_price = int(app.saved_cena_max)
except ValueError:
    wanted_price = 1e8

try:
    wanted_pow_min = int(app.saved_powierzchnia_od)
except ValueError:
    wanted_pow_min = 0

try:
    wanted_pow_max = int(app.saved_powierzchnia_do)
except ValueError:
    wanted_pow_max = 1000

try:
    wanted_rooms = int(app.saved_pokoje)
except ValueError:
    wanted_rooms = 1
pow = [0.8 * wanted_pow_min, 1.2 * wanted_pow_max]



In [69]:
print(wanted_lok)
print(wanted_price)
print(wanted_pow_min)
print(wanted_pow_max)
print(wanted_rooms)

Fabryczna
700000
45
1000
2


In [272]:
data_raw = data0.copy()
colnames = ['cena','powierzchnia']
replace_dict = {'zł': '', 'm2': '', '/': '', ' ': ''}

data_raw[colnames] = data_raw[colnames].astype('str')
data_raw[colnames] = data_raw[colnames].applymap(lambda x: str(x).replace('zł', '').replace('m²', '').replace('/', '').replace(' ', '').replace(',', '.'))
data_raw[colnames] = data_raw[colnames].astype('float')
data_raw['piętro'] = data_raw['piętro'].str.replace('parter', '0')
data_raw['liczba pięter'] = data_raw['piętro'].str.extract(r'(\d+)/(\d+)')[1].astype(float)
data_raw['piętro'] = data_raw['piętro'].str.extract(r'(\d+)').astype(float)
data_raw['pokoje'] = data_raw['pokoje'].fillna(1)
data_raw['rok budowy'] = 1969
data_raw = data_raw[((data_raw.cena < 1.1 * wanted_price) | data_raw.cena.isna()) & (data_raw.pokoje >= 2) & (data_raw.powierzchnia.between(pow[0], pow[1]))]
data_raw.isna().sum()

nazwa ogloszenia       0
cena                   0
opis                   0
lokalizacja            0
pokoje                 0
powierzchnia           0
rodzaj zabudowy       40
rynek                 40
piętro                41
stan wykonczenia      56
balkon/ogrod/taras     1
miejsce parkingowe    48
telefon               40
link                   0
rok budowy             0
liczba pięter         49
dtype: int64

In [273]:
dzielnice = '|'.join(['Fabryczna', 'Krzyki', 'Stare Miasto', 'Śródmieście', 'Psie Pole'])
data_raw['dzielnica'] = data_raw['lokalizacja'].str.extract(f'({dzielnice})')
data_raw['dzielnica'].fillna('Stare Miasto', inplace = True)
data_raw.insert(4, 'dzielnica', data_raw.pop('dzielnica'))
data_raw['rok budowy'] = 1969

data_raw['piętro'] = data_raw['piętro'].fillna(4)
data_raw['liczba pięter'] = data_raw[['liczba pięter', 'piętro']].max(axis=1).fillna(4)

column_moved = data_raw.columns[-1]
data_raw.insert(12, column_moved, data_raw.pop(column_moved))

data_raw['ulica'] = np.where(data_raw['lokalizacja'].str.contains('ul.'),
                             data_raw['lokalizacja'].str.extract(r'ul\.(.*?),', expand=False),
                             np.nan)

# Run the second line only for rows where 'ulica' is NaN
mask = data_raw['ulica'].isna()
data_raw.loc[mask, 'ulica'] = np.where(
    data_raw.loc[mask, 'nazwa ogloszenia'].str.contains('ul.'),
    data_raw.loc[mask, 'nazwa ogloszenia'].str.extract(r'ul\.(.*?),', expand=False),
    np.nan
)
data_raw.ulica = data_raw.ulica.str.strip()
data_raw['stan wykonczenia'].fillna('do remontu', inplace = True)
data_raw[['ulica', 'lokalizacja', 'nazwa ogloszenia']]
dzielnica_dict = {'Psie Pole': 'Zielna', 'Stare Miasto': 'Zamkowa', 'Fabryczna': 'Rudzka', 'Krzyki': 'Gajowa', 'Śródmieście': 'Kossaka'}
data_raw['ulica_dzielnica'] = data_raw['dzielnica'].map(dzielnica_dict)
data_raw['ulica'] = data_raw['ulica'].fillna(data_raw['ulica_dzielnica'])
data_raw['balkon/ogrod/taras'] = ~data_raw['balkon/ogrod/taras'].isna()
data_raw['miejsce parkingowe'] = ~data_raw['miejsce parkingowe'].isna()
wyk_dict = {'do remontu': 'Do remontu', 'do wykończenia': 'Do odświeżenia', 'do zamieszkania': 'Gotowe do wprowadzenia'}
data_raw['stan wykonczenia'] = data_raw['stan wykonczenia'].map(wyk_dict)
data_raw['stan wykonczenia'].fillna('Do remontu', inplace = True)

median_dict = data_raw.groupby('pokoje')['powierzchnia'].median().to_dict()
data_raw['powierzchnia'] = data_raw.apply(
    lambda row: row['powierzchnia'] if pd.notna(row['powierzchnia']) else median_dict.get(row['pokoje']),
    axis=1
)
data_raw0 = data_raw[['cena', 'ulica', 'pokoje', 'powierzchnia', 'piętro', 'liczba pięter', 'stan wykonczenia', 'balkon/ogrod/taras', 'miejsce parkingowe', 'rok budowy']]
data_raw0['ulica'].fillna('pl. Dominikański', inplace = True)
print(data_raw.shape)
data_raw.isna().sum()

(63, 19)


nazwa ogloszenia       0
cena                   0
opis                   0
lokalizacja            0
dzielnica              0
pokoje                 0
powierzchnia           0
rodzaj zabudowy       40
rynek                 40
piętro                 0
stan wykonczenia       0
balkon/ogrod/taras     0
liczba pięter          0
miejsce parkingowe     0
telefon               40
link                   0
rok budowy             0
ulica                  0
ulica_dzielnica        0
dtype: int64

In [274]:
model_data = data_raw0.copy()
model_data['stan wykonczenia'] = model_data['stan wykonczenia'].map({'Do remontu': 0, 'Do odświeżenia': 1, 'Gotowe do wprowadzenia': 2})
label_encoder = LabelEncoder()
model_data['ulica'] = label_encoder.fit_transform(model_data['ulica'])

In [275]:
loaded_model = load_model('model0')
try:
    predicted = predict_model(loaded_model, data=model_data[model_data.cena == 0])
    predicted.drop(columns=['cena'], inplace=True)
    pred_ids = predicted.index
    predicted.rename(columns={'prediction_label': 'cena'}, inplace=True)
    data_model = pd.concat([model_data[model_data.cena != 0], predicted])
    print('Predicted')
except ValueError:
    data_model = model_data.copy()
data_model = data_model[((data_model.cena < 1.2 * wanted_price) | data_model.cena.isna())]
data_model # tu sie cos jebie

Transformation Pipeline and Model Successfully Loaded
Predicted


,cena,ulica,pokoje,powierzchnia,piętro,liczba pięter,stan wykonczenia,balkon/ogrod/taras,miejsce parkingowe,rok budowy
2,629000.0000,11,2.0,50.000000,1.0,4.0,0,True,False,1969
4,506979.0000,10,3.0,51.470000,1.0,1.0,0,True,True,1969
5,699996.0000,11,3.0,60.000000,4.0,4.0,0,True,False,1969
8,488000.0000,8,3.0,44.000000,3.0,5.0,0,False,False,1969
11,610000.0000,1,2.0,38.400000,2.0,4.0,0,True,True,1969
12,662000.0000,0,2.0,45.000000,0.0,3.0,0,True,False,1969
13,426190.0000,10,2.0,37.060000,0.0,0.0,0,True,True,1969
14,575732.0000,2,4.0,68.950000,0.0,0.0,0,True,True,1969
16,515075.0000,6,2.0,43.320000,1.0,2.0,0,True,True,1969
17,673318.0000,6,3.0,58.170000,1.0,2.0,0,True,True,1969


In [276]:
loaded_model2 = load_model('model1')
new_prices = predict_model(loaded_model2, data = data_model)
new_prices['ulica'] = label_encoder.inverse_transform(new_prices['ulica'])
new_prices['stan wykonczenia'] = new_prices['stan wykonczenia'].map({0: 'Do remontu', 1: 'Do odświeżenia', 2: 'Gotowe do wprowadzenia'})
ids = new_prices.index
new_prices

Transformation Pipeline and Model Successfully Loaded


,ulica,pokoje,powierzchnia,piętro,liczba pięter,stan wykonczenia,balkon/ogrod/taras,miejsce parkingowe,rok budowy,cena,prediction_label
2,Zamkowa,2.0,50.000000,1.0,4.0,Do remontu,True,False,1969,629000.0000,6.809012e+05
4,Woskowa,3.0,51.470001,1.0,1.0,Do remontu,True,True,1969,506979.0000,5.380191e+05
5,Zamkowa,3.0,60.000000,4.0,4.0,Do remontu,True,False,1969,699996.0000,7.217018e+05
8,Rudzka,3.0,44.000000,3.0,5.0,Do remontu,False,False,1969,488000.0000,6.948391e+05
11,Gajowa,2.0,38.400002,2.0,4.0,Do remontu,True,True,1969,610000.0000,7.296614e+05
12,Agrestowa,2.0,45.000000,0.0,3.0,Do remontu,True,False,1969,662000.0000,6.716639e+05
13,Woskowa,2.0,37.060001,0.0,0.0,Do remontu,True,True,1969,426190.0000,4.248255e+05
14,Ignacego Mościckiego,4.0,68.949997,0.0,0.0,Do remontu,True,True,1969,575732.0000,7.618489e+05
16,Nefrytowa,2.0,43.320000,1.0,2.0,Do remontu,True,True,1969,515075.0000,5.826168e+05
17,Nefrytowa,3.0,58.169998,1.0,2.0,Do remontu,True,True,1969,673318.0000,7.887932e+05


In [277]:
to_rate = data_raw[['dzielnica', 'pokoje', 'powierzchnia', 'stan wykonczenia', 'balkon/ogrod/taras', 'miejsce parkingowe', 'cena']]
to_rate = to_rate.join(new_prices['prediction_label'])#.loc[ids]
to_rate

,dzielnica,pokoje,powierzchnia,stan wykonczenia,balkon/ogrod/taras,miejsce parkingowe,cena,prediction_label
2,Stare Miasto,2.0,50.00,Do remontu,True,False,629000.0,680901.2500
4,Krzyki,3.0,51.47,Do remontu,True,True,506979.0,538019.1250
5,Stare Miasto,3.0,60.00,Do remontu,True,False,699996.0,721701.8125
8,Fabryczna,3.0,44.00,Do remontu,False,False,488000.0,694839.0625
11,Krzyki,2.0,38.40,Do remontu,True,True,610000.0,729661.3750
...,...,...,...,...,...,...,...,...
150,Krzyki,2.0,39.48,Do remontu,True,False,0.0,647987.1875
213,Krzyki,5.0,89.36,Do remontu,True,False,0.0,NaN
240,Stare Miasto,2.0,36.03,Do remontu,True,False,0.0,567955.9375
242,Stare Miasto,3.0,58.59,Do remontu,True,False,0.0,720730.7500


In [278]:
to_rate = data_raw[['dzielnica', 'pokoje', 'powierzchnia', 'stan wykonczenia', 'balkon/ogrod/taras', 'miejsce parkingowe', 'cena']]
to_rate = to_rate.join(new_prices['prediction_label'])

if wanted_lok != '':
    to_rate['wanted_loc'] = to_rate['dzielnica'] == wanted_lok
else:
    to_rate['wanted_loc'] = True

to_rate['pokoje_bon'] = to_rate['pokoje'] > wanted_rooms
to_rate['stan wykonczenia'] = to_rate['stan wykonczenia'].map({'Do remontu': 0, 'Do odświeżenia': 1, 'Gotowe do wprowadzenia': 2})
to_rate

,dzielnica,pokoje,powierzchnia,stan wykonczenia,balkon/ogrod/taras,miejsce parkingowe,cena,prediction_label,wanted_loc,pokoje_bon
2,Stare Miasto,2.0,50.00,0,True,False,629000.0,680901.2500,False,False
4,Krzyki,3.0,51.47,0,True,True,506979.0,538019.1250,False,True
5,Stare Miasto,3.0,60.00,0,True,False,699996.0,721701.8125,False,True
8,Fabryczna,3.0,44.00,0,False,False,488000.0,694839.0625,True,True
11,Krzyki,2.0,38.40,0,True,True,610000.0,729661.3750,False,False
...,...,...,...,...,...,...,...,...,...,...
150,Krzyki,2.0,39.48,0,True,False,0.0,647987.1875,False,False
213,Krzyki,5.0,89.36,0,True,False,0.0,NaN,False,True
240,Stare Miasto,2.0,36.03,0,True,False,0.0,567955.9375,False,False
242,Stare Miasto,3.0,58.59,0,True,False,0.0,720730.7500,False,True


In [279]:
weights = {
    'pokoje': 2,
    'powierzchnia': 2,
    'stan wykonczenia': 2,
    'balkon/ogrod/taras': 1,
    'miejsce parkingowe': 1,
    'cena': -4,  
    'prediction_label': -2,
    'wanted_loc': 3,
    'pokoje_bon': 2
}

numeric_columns = to_rate.select_dtypes(include=['number']).columns
normalized_data = to_rate.copy()
for column in numeric_columns:
    if (to_rate[column].max() - to_rate[column].min()) != 0:
        normalized_data[column] = (to_rate[column] - to_rate[column].min()) / (to_rate[column].max() - to_rate[column].min())
    else:
        normalized_data[column] = 1

normalized_data['Ocena'] = sum(normalized_data[column] * weights[column] for column in weights)
score_min = normalized_data['Ocena'].min()
score_max = normalized_data['Ocena'].max()
normalized_data['Ocena'] = round(((normalized_data['Ocena'] - score_min) * (10 / (score_max - score_min))),2)
best = normalized_data[['dzielnica', 'Ocena']].sort_values('Ocena', ascending=False)[:7]
best

,dzielnica,Ocena
32,Fabryczna,10.00
20,Fabryczna,9.75
84,Psie Pole,9.27
72,Psie Pole,9.20
61,Krzyki,9.18
85,Krzyki,8.76
64,Stare Miasto,8.62


In [284]:
used_cols = ['nazwa ogloszenia', 'cena', 'lokalizacja', 'dzielnica', 'ulica', 'pokoje', 'powierzchnia', 'rodzaj zabudowy','telefon', 'link', 'Ocena']
final = data.join(best, how = 'right')
final['ulica'] = np.where(final['lokalizacja'].str.contains('ul.'),
                             final['lokalizacja'].str.extract(r'ul\.(.*?),', expand=False),
                             np.nan)

# Run the second line only for rows where 'ulica' is NaN
mask = final['ulica'].isna()
final.loc[mask, 'ulica'] = np.where(
    final.loc[mask, 'nazwa ogloszenia'].str.contains('ul.'),
    final.loc[mask, 'nazwa ogloszenia'].str.extract(r'ul\.(.*?),', expand=False),
    np.nan
)
final.ulica = final.ulica.str.strip()
dzielnice = '|'.join(['Fabryczna', 'Krzyki', 'Stare Miasto', 'Śródmieście', 'Psie Pole'])
final['dzielnica'] = final['lokalizacja'].str.extract(f'({dzielnice})')
final.insert(4, 'dzielnica', final.pop('dzielnica'))
final.insert(5, 'ulica', final.pop('ulica'))
dzielnica_dict = {'Psie Pole': 'Zielna', 'Stare Miasto': 'Zamkowa', 'Fabryczna': 'Rudzka', 'Krzyki': 'Gajowa', 'Śródmieście': 'Kossaka'}
final['ulica_dzielnica'] = final['dzielnica'].map(dzielnica_dict)
final['ulica'] = final['ulica'].fillna(final['ulica_dzielnica'])
final['ulica'] = final['ulica'] + ', Wrocław'


In [285]:
final[used_cols]
#final.to_csv('sample4map3.csv')

,nazwa ogloszenia,cena,lokalizacja,dzielnica,ulica,pokoje,powierzchnia,rodzaj zabudowy,telefon,link,Ocena
32,Widok na zieleń | Ostatnie mieszkania,724 259 zł,"ul. Maślicka, Maślice, Fabryczna, Wrocław, dol...",Fabryczna,"Maślicka, Wrocław",4.0,"71,75 m²",blok,602 101 602,https://www.otodom.pl/pl/oferta/widok-na-ziele...,10.00
20,3 pokoje z 2 tarasami/gotowe do odbioru,679 780 zł,"ul. Jordanowska, Oporów, Fabryczna, Wrocław, d...",Fabryczna,"Jordanowska, Wrocław",3.0,"58,1 m²",blok,602 101 602,https://www.otodom.pl/pl/oferta/3-pokoje-z-2-t...,9.75
84,Opis nieruchomości,NaN,"Psie Pole,",Psie Pole,"Zielna, Wrocław",4.0,"72,96 m²",NaN,NaN,https://www.morizon.pl/oferta/sprzedaz-mieszka...,9.27
72,Opis nieruchomości,NaN,"Psie Pole,",Psie Pole,"Zielna, Wrocław",4.0,"69,51 m²",NaN,NaN,https://www.morizon.pl/oferta/sprzedaz-mieszka...,9.20
61,Opis nieruchomości,NaN,"Krzyki,",Krzyki,"Gajowa, Wrocław",4.0,"71,41 m²",NaN,NaN,https://www.morizon.pl/oferta/sprzedaz-mieszka...,9.18
85,Opis nieruchomości,NaN,"Krzyki,",Krzyki,"Gajowa, Wrocław",3.0,"61,23 m²",NaN,NaN,https://www.morizon.pl/oferta/sprzedaz-mieszka...,8.76
64,Opis nieruchomości,NaN,"Tarnogaj,",NaN,NaN,3.0,"59,53 m²",NaN,NaN,https://www.morizon.pl/oferta/sprzedaz-mieszka...,8.62


In [286]:
idd = final[final.cena.isna()].index
final['cena'].fillna(round(predicted.loc[idd].cena, 0).astype(int), inplace = True)
final['cena'] = final['cena'].astype(str).str.strip('.0')
final.loc[~final['cena'].str.contains(' zł'), 'cena'] = final['cena'] + ' zł*'
#final.loc[final['cena'].str.contains('\*'), 'cena'] = final['cena'][:3] + ' ' + final['cena'][3:]


In [287]:
final[used_cols]

,nazwa ogloszenia,cena,lokalizacja,dzielnica,ulica,pokoje,powierzchnia,rodzaj zabudowy,telefon,link,Ocena
32,Widok na zieleń | Ostatnie mieszkania,724 259 zł,"ul. Maślicka, Maślice, Fabryczna, Wrocław, dol...",Fabryczna,"Maślicka, Wrocław",4.0,"71,75 m²",blok,602 101 602,https://www.otodom.pl/pl/oferta/widok-na-ziele...,10.00
20,3 pokoje z 2 tarasami/gotowe do odbioru,679 780 zł,"ul. Jordanowska, Oporów, Fabryczna, Wrocław, d...",Fabryczna,"Jordanowska, Wrocław",3.0,"58,1 m²",blok,602 101 602,https://www.otodom.pl/pl/oferta/3-pokoje-z-2-t...,9.75
84,Opis nieruchomości,837607 zł*,"Psie Pole,",Psie Pole,"Zielna, Wrocław",4.0,"72,96 m²",NaN,NaN,https://www.morizon.pl/oferta/sprzedaz-mieszka...,9.27
72,Opis nieruchomości,801924 zł*,"Psie Pole,",Psie Pole,"Zielna, Wrocław",4.0,"69,51 m²",NaN,NaN,https://www.morizon.pl/oferta/sprzedaz-mieszka...,9.20
61,Opis nieruchomości,809416 zł*,"Krzyki,",Krzyki,"Gajowa, Wrocław",4.0,"71,41 m²",NaN,NaN,https://www.morizon.pl/oferta/sprzedaz-mieszka...,9.18
85,Opis nieruchomości,756532 zł*,"Krzyki,",Krzyki,"Gajowa, Wrocław",3.0,"61,23 m²",NaN,NaN,https://www.morizon.pl/oferta/sprzedaz-mieszka...,8.76
64,Opis nieruchomości,756532 zł*,"Tarnogaj,",NaN,NaN,3.0,"59,53 m²",NaN,NaN,https://www.morizon.pl/oferta/sprzedaz-mieszka...,8.62
